In [29]:
import pandas as pd, numpy as np
from db_queries import get_ids, get_outputs, get_location_metadata, get_covariate_estimates, get_population
from get_draws.api import get_draws
import scipy.stats
from operator import attrgetter
from math import sqrt, pi
import matplotlib.pyplot as plt

# The goal of this notebook is to try to recreate the maternal disorder/iron deficiency PAF using the maternal YLD and YLL rates and iron deficiency risk exposure

The steps of this analysis will include:

1) Load risk exposure, TMREL, relative risks, and maternal YLD and YLL rates

2) Calculate PAF

3) Compare to burdenator pulled PAF

In [2]:
# define location and age group IDs of interest

locs = [165, 161, 214, 182] # pakistan, bangladesh, nigeria, malawi
ages = list(range(7,16))
# reminder, always want sex_id=2 (female)

# 1. Load relevant parameters

In [8]:
rr = get_draws('rei_id',
                 95,
                 source='rr',
                 location_id=1,
                 age_group_id=ages,
                 sex_id=2,
                 year_id=2019,
                 gbd_round_id=6,
                 decomp_step='step4'
                 )
rr.head()

# note, this dataframe is age- and cause-specific, but values are
# identical across all ages and causes
# (all causes are most detailed within maternal disorders)

,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,cause_id,mortality,morbidity,metric_id,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
0,95,9024,1,2019,8,2,367,1,1,3,...,1.189681,1.315712,1.28604,1.257695,1.216908,1.177559,1.315347,1.221176,1.276526,1.243538
1,95,9024,1,2019,13,2,367,1,1,3,...,1.189681,1.315712,1.28604,1.257695,1.216908,1.177559,1.315347,1.221176,1.276526,1.243538
2,95,9024,1,2019,11,2,367,1,1,3,...,1.189681,1.315712,1.28604,1.257695,1.216908,1.177559,1.315347,1.221176,1.276526,1.243538
3,95,9024,1,2019,10,2,367,1,1,3,...,1.189681,1.315712,1.28604,1.257695,1.216908,1.177559,1.315347,1.221176,1.276526,1.243538
4,95,9024,1,2019,9,2,367,1,1,3,...,1.189681,1.315712,1.28604,1.257695,1.216908,1.177559,1.315347,1.221176,1.276526,1.243538


In [10]:
ylds = get_draws('cause_id',
                 367,
                 source='como',
                 location_id=locs,
                 age_group_id=ages,
                 measure_id=3,
                 sex_id=2,
                 year_id=2019,
                 gbd_round_id=6,
                 decomp_step='step5'
                 )
# note: metric_id=3 (rate)
ylds.head()

,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sex_id,year_id,metric_id
0,7,367,7.283970e-07,0.000001,9.530025e-07,7.555435e-07,5.293502e-07,3.025573e-07,5.266001e-07,6.819117e-07,...,7.720953e-07,2.959184e-07,5.424846e-07,6.748289e-07,5.303953e-07,161,3,2,2019,3
1,8,367,1.697296e-04,0.000140,1.151586e-04,9.308121e-05,1.137195e-04,7.442830e-05,1.098248e-04,1.086779e-04,...,1.418403e-04,6.722950e-05,1.365856e-04,7.942999e-05,8.514004e-05,161,3,2,2019,3
2,9,367,2.276003e-04,0.000203,3.626974e-04,2.938739e-04,2.917058e-04,2.405824e-04,2.174652e-04,2.147612e-04,...,2.560316e-04,8.726840e-05,2.560195e-04,1.671378e-04,1.504830e-04,161,3,2,2019,3
3,10,367,2.042305e-04,0.000180,2.385139e-04,3.686240e-04,3.385269e-04,2.888382e-04,3.272004e-04,2.046238e-04,...,2.892034e-04,1.170274e-04,1.603473e-04,2.451001e-04,1.736127e-04,161,3,2,2019,3
4,11,367,1.279405e-04,0.000150,7.894112e-05,1.495101e-04,1.514786e-04,1.126843e-04,2.642369e-04,1.635994e-04,...,1.953879e-04,5.414984e-05,1.096046e-04,1.720946e-04,1.665320e-04,161,3,2,2019,3


In [32]:
yll_count = get_draws('cause_id',
                 367,
                 source='codcorrect',
                 location_id=locs,
                 age_group_id=ages,
                 sex_id=2,
                 measure_id=4,
                 year_id=2019,
                 gbd_round_id=6,
                 decomp_step='step5',
                 status='latest')
# note: metric_id=1 (NUMBER)
yll_count.head()

,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sex_id,year_id,metric_id
0,7,367,3068.047648,2364.327119,3922.304359,923.794922,1670.581500,1656.338574,3138.454117,1803.426682,...,804.348015,1819.398528,2005.979635,4828.300725,2945.854187,161,4,2,2019,1
1,8,367,18962.348882,21596.048676,21293.496575,16910.901854,15743.915644,16241.869392,20120.567835,18655.726674,...,20533.561141,13144.864341,23042.334383,25016.081946,25898.720461,161,4,2,2019,1
2,9,367,25164.107224,22388.625373,29331.724218,20420.268860,25111.758872,18465.592966,23704.972885,31346.072690,...,30321.996209,19828.804040,36424.115556,33251.534751,20521.905058,161,4,2,2019,1
3,10,367,37898.097233,23710.646321,35147.700853,24373.717461,24807.059235,21534.316597,20312.623054,31217.164695,...,33818.676992,24729.399673,32485.010665,22935.644502,24126.705045,161,4,2,2019,1
4,11,367,27433.664515,27426.465759,24171.913543,18920.636280,15188.663063,18226.329983,19295.105818,19555.936627,...,28649.748059,17567.525586,23441.172139,21935.538490,13213.505474,161,4,2,2019,1


In [49]:
# get population

pop = get_population(location_id=locs,
                    age_group_id=ages,
                    sex_id=2,
                    year_id=2019,
                    gbd_round_id=6,
                    decomp_step='step5')
pop.head()

,age_group_id,location_id,year_id,sex_id,population,run_id
0,7,161,2019,2,7.695495e+06,192
1,8,161,2019,2,7.921423e+06,192
2,9,161,2019,2,7.756269e+06,192
3,10,161,2019,2,7.297086e+06,192
4,11,161,2019,2,6.586882e+06,192


In [45]:
len(ylls_prepped) /4

9.0

In [51]:
pop['population'].values

array([ 7695495.34070275,  7921423.23144917,  7756268.6842713 ,
        7297085.58742806,  6586881.88946491,  5723006.46624669,
        5181219.40311745,  4427082.24085176,  3638801.79010677,
       13290613.45213349, 11977326.59627261, 10367170.61586668,
        8697046.66104244,  7472130.19589056,  6503358.01363847,
        5590714.87389896,  4623748.12084878,  3487373.26313745,
        1319886.64181976,  1168700.85024049,   908328.74207117,
         729597.00735055,   640923.49463646,   525547.76131381,
         388481.80308714,   292961.1875441 ,   215275.74524477,
       14875508.58541586, 12603869.94843236, 10472197.98981012,
        8607420.83511819,  7046190.4503177 ,  5814476.98021114,
        4813949.0373193 ,  4059780.86865446,  3374481.37590223])

In [50]:
# calculate yll RATE

ylls_prepped = yll_count.set_index([c for c in yll_count.columns if 'draw' not in c])
ylls = ylls_prepped / pop['population'].values
ylls.head()

ValueError: Unable to coerce to Series, length must be 1000: given 36

In [ ]:
#ylls_prepped = (yll_count
#                .set_index(['age_group_id','location_id','year_id','sex_id'])
#                .drop(columns=['measure_id','metric_id','cause_id']))
#pop_prepped = (pop.set_index(['age_group_id','location_id','year_id','sex_id'])
#               .drop(columns=['run_id']))
#ylls = ylls_prepped / pop['population'].values
#ylls.head()

In [25]:
# get TMREL

In [26]:
# get risk exposure

# Calculate PAF

In [27]:
# calculate PAF

# Compare calculated to pulled PAF

In [28]:
# pull paf from burdenator
burdenator_pafs = get_draws(['cause_id','rei_id'],
                            [367,95],
                            source='burdenator',
                            location_id=locs,
                            measure_id=[3,4],
                            metric_id=2,
                            year_id=[2019],
                            gbd_round_id=6,
                            decomp_step='step5',
                            status='best')
burdenator_pafs.head()

,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,metric_id,rei_id,sex_id,year_id
0,7,367,0.111996,0.140040,0.169013,0.154807,0.078587,0.125657,0.103909,0.096825,...,0.101085,0.099284,0.134437,0.095228,161,3,2,95,2,2019
1,8,367,0.184341,0.192106,0.201641,0.244485,0.135061,0.215163,0.142851,0.127113,...,0.238394,0.157278,0.183848,0.152484,161,3,2,95,2,2019
2,9,367,0.172627,0.286573,0.241478,0.273874,0.145159,0.261510,0.152150,0.149250,...,0.201589,0.170215,0.200242,0.160986,161,3,2,95,2,2019
3,10,367,0.138526,0.220173,0.228901,0.219605,0.122970,0.247734,0.172014,0.114019,...,0.205065,0.148081,0.202039,0.148174,161,3,2,95,2,2019
4,11,367,0.146134,0.267832,0.214096,0.194884,0.121695,0.184113,0.185716,0.138453,...,0.173657,0.147022,0.193002,0.196635,161,3,2,95,2,2019
